Input

In [1]:
file_path = "input.txt"  # Path to the input file

with open(file_path, "r") as f:
    data = f.read()

Part 1

In [44]:
warehouse, moves = data.split('\n\n')
warehouse = [[x for x in row]for row in warehouse.split('\n')]
moves = ''.join(moves.split('\n'))

m = len(warehouse)
n = len(warehouse[0])

move_map = {
    '^': (-1, 0),
    'v': (1, 0),
    '<': (0, -1),
    '>': (0, 1)
}

# get robot initial position
robot_pos = (0, 0)
for i in range(m):
    for j in range(n):
        if warehouse[i][j] == '@':
            robot_pos = (i, j)
            break

# simulate grid
for move in moves:
    move_dir = move_map[move]
    ni, nj = robot_pos[0] + move_dir[0], robot_pos[1] + move_dir[1]
    if warehouse[ni][nj] == '.':
        warehouse[ni][nj] = '@'
        warehouse[robot_pos[0]][robot_pos[1]] = '.'
        robot_pos = (ni, nj)
        continue
    elif warehouse[ni][nj] == '#':
        continue
    
    while warehouse[ni][nj] == 'O':
        ni, nj = ni + move_dir[0], nj + move_dir[1]

    if warehouse[ni][nj] == '#': 
        continue # can't move blocked
    else: # move all boxes!
        while (ni, nj) != robot_pos:
            warehouse[ni][nj] = warehouse[ni-move_dir[0]][nj-move_dir[1]]
            ni -= move_dir[0]
            nj -= move_dir[1]
        warehouse[robot_pos[0]][robot_pos[1]] = '.'
        robot_pos = (robot_pos[0] + move_dir[0], robot_pos[1] + move_dir[1])

print('\n'.join(''.join(row) for row in warehouse))

score = 0
for i in range(m):
    for j in range(n):
        if warehouse[i][j] == 'O':
            score += i*100 + j

print(score)

#######
#@..#.#
#.O...#
#..O..#
#..O..#
#.....#
#######
908


Part 2

In [2]:
from IPython.display import clear_output
from time import sleep

warehouse, moves = data.split('\n\n')
grid = [[x for x in row]for row in warehouse.split('\n')]
moves = ''.join(moves.split('\n'))

n = len(grid)

move_map = {
    '^': (-1, 0),
    'v': (1, 0),
    '<': (0, -1),
    '>': (0, 1)
}

# get robot initial position
boxes = []
walls = []
for i in range(n):
    for j in range(n):
        if grid[i][j] == '@':
            ci, cj = i, j*2
        elif grid[i][j] == '#':
            walls.append([i, j*2])
            walls.append([i, j*2+1])
        elif grid[i][j] == 'O':
            boxes.append([i, j*2])

def print_grid(boxes, walls, robot_pos):
    global m, n
    grid = [['.']*2*n for _ in range(n)]

    for box in boxes:
        grid[box[0]][box[1]] = '['
        grid[box[0]][box[1] + 1] = ']'
    
    for wall in walls:
        grid[wall[0]][wall[1]] = '#'

    grid[robot_pos[0]][robot_pos[1]] = '@'
    # clear_output()
    print('\n'.join(''.join(row) for row in grid))
    # sleep(0.08)

def in_grid(i, j):
    return (0 <= i < n) and (0 <= j < 2*n)

def move(dir):
    global ci, cj, warehouse
    newi, newj = ci + dir[0], cj + dir[1]
    # if not in_grid(newi, newj): return

    if [newi, newj] in walls: return

    boxes_to_move = []
    if [newi, newj] in boxes: boxes_to_move.append([newi, newj])
    if [newi, newj-1] in boxes: boxes_to_move.append([newi, newj-1])

    blocked = False
    seen = set()

    while len(boxes_to_move) > 0:
        ti, tj = boxes_to_move.pop()
        ni, nj = ti + dir[0], tj + dir[1]

        # if not in_grid(ni, nj):
        #     blocked = True
        #     break
        if [ni, nj] in walls or [ni, nj+1] in walls:
            blocked = True
            break
        if (ti, tj) in seen: continue
        seen.add((ti, tj))

        if [ni, nj] in boxes: boxes_to_move.append([ni, nj])
        if [ni, nj+1] in boxes: boxes_to_move.append([ni, nj+1])
        if [ni, nj-1] in boxes: boxes_to_move.append([ni, nj-1])
    
    if blocked: return

    for i, box in enumerate(boxes):
        if tuple(box) in seen:
            boxes[i][0] += dir[0]
            boxes[i][1] += dir[1]

    ci += dir[0]
    cj += dir[1]

for dir in moves:
    move(move_map[dir])
    # print_grid(boxes, walls, (ci, cj))

score = 0
for box in boxes: score += box[0]*100 + box[1]
score

1576353